In [1]:
import surprise

print(surprise.__version__)

1.1.3


In [10]:
!pip upgrade numpy

ERROR: unknown command "upgrade"



In [5]:
!pip uninstall numpy

^C


In [6]:
!pip install numpy


[notice] A new release of pip available: 22.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터 분할 -> random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\admin/.surprise_data/ml-100k


In [3]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [4]:
predictions=algo.test(testset)
print('prediction type : ',type(predictions), 'size : ', len(predictions))
print('prediction 경과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'> size :  25000
prediction 경과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

In [5]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [6]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [7]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

### Surprise 주요 모듈 소개

In [8]:
import pandas as pd

ratings=pd.read_csv('ml-latest-small/ratings.csv')
ratings.to_csv('ml-latest-small/ratings_noh.csv', index=False, header=False)

In [10]:
from surprise import Reader
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('./ml-latest-small/ratings_noh.csv', reader=reader)

In [12]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 -> 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions=algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [13]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

# ratings DF 에서 컬럼 순서 지켜야 함
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
trainset, testeset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.0388


1.0387858758781252

### 교차 검증과 하이퍼 파라미터 튜닝

In [14]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8753  0.8718  0.8738  0.8799  0.8718  0.8745  0.0030  
MAE (testset)     0.6724  0.6701  0.6692  0.6757  0.6720  0.6718  0.0022  
Fit time          0.77    0.80    0.87    0.84    0.79    0.81    0.04    
Test time         0.19    0.11    0.19    0.11    0.17    0.15    0.04    


{'test_rmse': array([0.87531194, 0.8717503 , 0.87381988, 0.87993851, 0.87183855]),
 'test_mae': array([0.67235141, 0.67008769, 0.66915155, 0.6756574 , 0.67195604]),
 'fit_time': (0.7669870853424072,
  0.8030600547790527,
  0.8680603504180908,
  0.8369286060333252,
  0.7940719127655029),
 'test_time': (0.1860055923461914,
  0.10500526428222656,
  0.19394683837890625,
  0.10706615447998047,
  0.17099523544311523)}

In [15]:
from surprise.model_selection import GridSearchCV

# 최적화 파라미터 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20,40,60], 'n_factors': [50,100,200]}

# CV=3, 성능 평가 rmse, mse로 수행
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'],cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8770335387512077
{'n_epochs': 20, 'n_factors': 50}


### Surprise를 이용한 개인화 영화 추천 시스템 구축

In [17]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [18]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [19]:
# 영화에 대한 상세 속성 정보 df 로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# uesrId=9의 movieId 데이터를 추출 movieId=42 데이터 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [20]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [23]:
def get_unseen_surprise(ratings, movies, userId):
    # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    
    # 모든 영화의 movieId를 리스트로 생성
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화의 movieId 중 이미 평점을 매긴 영화 제외한 후 리스트 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수: ', len(seen_movies), '추천 대상 영화 수: ', len(unseen_movies), '전체 영화 수: ', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수:  46 추천 대상 영화 수:  9696 전체 영화 수:  9742


In [25]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    
    # 알고리즘 객체의 predict() 메서드 평점 X 영화에 반복 수행 -> 결과 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # prediction 리스트 객체 -> surprise Predictions 객체 원소로 가지고 있음
    # sortkey_est 함수 정의 -> list 객체 sort() 함수
    def sortkey_est(pred):
        return pred.est
    
    # 내림차순 정렬 수행, top_n개 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    
    # 추출된 영화 정보 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('-------TOP 10 추천 영화 리스트-------')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수:  46 추천 대상 영화 수:  9696 전체 영화 수:  9742
-------TOP 10 추천 영화 리스트-------
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


#### 딥러닝 기반 추천시스템

https://velog.io/@tobigs-recsys/DL-based-Recommender-Systems-%EB%94%A5%EB%9F%AC%EB%8B%9D-%EA%B8%B0%EB%B0%98-%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C
